# Написание приложения с использованием функционального подхода

### 1. Техническое задание
### Название приложения: **Учет финансов**
### Функционал:
1. Приложение для одного пользователя
2. В приложение можно заносить поступления и траты финансов
3. Можно задавать категорию
4. Можно задавать конкретную дату
5. Можно получать статистику за определенный промежуток времени 
6. В статистике отображаются траты и поступления по категориям

### 2. Какие библиотекии используем?

In [14]:
import datetime as dt
from collections import defaultdict
import json
import os.path

### 3. Пользовательский интерфейс

In [2]:
def greet():
    print("Приветствуем вас в приложении")
    print("-----------------------------")
    print("------- Учёт финансов -------")
    print("-----------------------------")
    print("        Режимы работы:       ")
    print("-----------------------------")
    print("   1.  Добавить зачисление   ")
    print("   2.  Добавить трату        ")
    print("   3.  Получить статистику   ")
    print("   4.  Выйти из программы    ")
    print("-----------------------------")
    print(" Формат ввода даты: YY-MM-DD ")

def ask_mode():
    print("-----------------------------")
    return input("    Выберите режим работы:   ")

def wrong():
    print("-----------------------------")
    print(" Вы ввели некорректный режим ")

In [47]:
def ask_cost(ask_str):
    number = None
    while number is None:
        num_str =  input("  Введите сумму:             ")
        if num_str.isdigit():
            number = int(num_str)
        else:
            print(" Вы ввели некорректное число!")
            continue
    return number

def ask_date(ask_str):
    date = None
    while date is None:
        date_str = input(ask_str)
        splitted = date_str.split("-")
        
        if len(splitted)!= 3:
            print(" Введена некорректная дата!  ")
            continue
        
        check = [] 
        for string in splitted:
            check.append(string.isdigit())
        
        if not(all(check)):
            print(" Часть даты- не число!       ")
            continue
        
        splitted[0] = "20"+splitted[0]
        year, month, day = map(int, splitted)
        date = dt.date(year, month, day)
    return date

In [18]:
def ask_income():
    print("-----------------------------")
    cat = input("  Введите категорию:         ")
    cost = ask_cost("  Введите сумму поступления:")
    date = ask_date("  Введите дату поступления: ")
    return ["+",cat, date, cost]

def ask_spend():
    print("-----------------------------")
    cat = input("  Введите категорию:         ")
    cost = ask_cost("  Введите сумму траты:      ")
    date = ask_date("  Введите дату траты:       ")
    return ["-",cat, date, cost]

### 4. Вывод статистики

In [ ]:
def stat_all_time(data):
    income_stat = defaultdict(list)
    spend_stat = defaultdict(list)
    
    for typ, cat, date, cost in data:
        if typ == "+":
            income_stat[cat].append(cost)
        if typ == "-":
            spend_stat[cat].append(cost)
    
    
    print("-----------------------------")
    print("  Статистика поступлений:    ")
    print("-----------------------------")
    
    for cat, cost_list in income_stat.items():
        print(f"  {cat:20} - {sum(cost_list)}")
    
    print("-----------------------------")
    print("  Статистика трат:           ")
    print("-----------------------------")
    for cat, cost_list in spend_stat.items():
        print(f"  {cat:20} - {sum(cost_list)}")

In [27]:
def ask_interval():
    date1 = ask_date("  Введите начало интервала:")
    date2 = ask_date("  Введите конец интервала: ")
    return (date1,date2)

def stat(data, date1, date2):
    income_stat = defaultdict(list)
    spend_stat = defaultdict(list)
    
    for typ, cat, date, cost in data:
        if date >= date1 and date <= date2:
            if typ == "+":
                income_stat[cat].append(cost)
            if typ == "-":
                spend_stat[cat].append(cost)
    
    print("-----------------------------")
    print("  Статистика поступлений:    ")
    print("-----------------------------")
    for cat, cost_list in income_stat.items():
        print(f"  {cat:20} - {sum(cost_list)}")
    print("-----------------------------")
    print("  Статистика трат:           ")
    print("-----------------------------")
    for cat, cost_list in spend_stat.items():
        print(f"  {cat:20} - {sum(cost_list)}")

### 5. Цикл с опросом пользователя

In [76]:
def loop(data):
    greet()
    
    while True:
        mode = ask_mode()
        if mode == "1":
            data.append(ask_income())
        elif mode == "2":
            data.append(ask_spend())
        elif mode == "3":
            beg, end = ask_interval()
            stat(data, beg, end)
        elif mode == "4":
            break
        else:
            wrong()
    
    print("-----------------------------")
    print("      Данные сохранены!      ")
    print("-----------------------------")
    
    return data


In [77]:
loop([])

Приветствуем вас в приложении
-----------------------------
------- Учёт финансов -------
-----------------------------
        Режимы работы:       
-----------------------------
   1.  Добавить зачисление   
   2.  Добавить трату        
   3.  Получить статистику   
   4.  Выйти из программы    
-----------------------------
 Формат ввода даты: YY-MM-DD 
-----------------------------


    Выберите режим работы:    4


-----------------------------
      Данные сохранены!      
-----------------------------


[]

### 6. Работа с файлами

In [64]:
def save(data, FILENAME):
    for rec in data:
        rec[2] = str(rec[2])
    with open(FILENAME, "w") as f:
        json.dump(data, f)

def read(FILENAME):
    if os.path.isfile(FILENAME):
        with open(FILENAME) as f:
            data = json.load(f)
    else:
        data = []
    
    for rec in data:
        date_ints = map(int, rec[2].split("-"))
        rec[2] = dt.date(*date_ints)
    
    return data

### 7. Собираем все вместе

In [ ]:
def main(FILENAME):
    data = read(FILENAME)
    data = loop(data)
    save(data, FILENAME)

In [ ]:
main("journal.json")

In [73]:
def ask_file(FILENAME):
    print("-----------------------------")
    print(" Путь к файлу по-умолчанию:  ")
    print(f" {FILENAME}")
    print("-----------------------------")
    print("  Нажмите Enter для работы   ")
    print("       с этим файлом.        ")
    print("  Введите имя файла, если    ")
    print("  нужно работать с другим    ")
    command = input()
    if command:
        if os.path.isfile(command):
            FILENAME = command
            print("-----------------------------")
            print("  Файл выбран!     ")
        else:
            print("-----------------------------")
            print("  Такого файла нет!    ")
            print("  Использую файл по-умолчанию    ")
    return FILENAME
        

### 8. Домашнее задание
1. Статистика по всему промежутку времени
2. Подсчет общего количества трат и поступлений
3. Статистика общих трат и поступлений по нескольким месяцам
4. Аналогично пункту только по какой-то категории
5. Программа для нескольких пользователей
6. \* Шифровать данные каждого пользователя паролем